# 네이버 매장 존재 여부 확인 크롤링

문제 
1. 강남구 매장이지만, 네이버로 검색하는 경우 강남구가 아닌 다른 지역의 매장명이 검색됨
2. 데이터 반영이 빠른편이지만, 네이버에 매장이 없는 경우가 있음 (없는 매장 : 맥도날드, 교촌, 굽네) 
3. 검색하면 전혀 다른 업체가 나오는 경우가 있음 -> 네이버 업태구분명을 크롤링해와서 참고하여 반영해야 
4. 나라에서 제공하는 data를 사용하는 경우 인기도를 알기 어려움. 아닌가. -> 네이버 지도 api를 사용해서 점수를 부여할까? 
5. 자료 업데이트 시기 문제 - data를 다운로드 받아서 사용하는 경우, 매장 정보는 계속 바뀐는데 업데이트가 늦잖아... 매장 정보의 변동이 생기 사람이 계속 업로드를 해야 하잖아.. 

#### 자료 
https://www.localdata.go.kr/devcenter/dataDown.do?menuNo=20001

1. 관광식당, 일반음식점, 휴게음식점 데이터셋 다운로드(07_24_01_P_.csv.zip, 07_24_04_P_.csv.zip , 07_24_05_P_.csv.zip)
2. 다운로드한 파일들을 data 폴더에 넣어주세요. (압축풀지말고)

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 

import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os


from selenium import webdriver
import chromedriver_autoinstaller
import webbrowser

## 매장 데이터 다운로드 후 전처리

In [4]:
# zip 파일 압축 풀기 

folder_path = './data'
zip_folder = './data/zip_data'

# data 폴더 안에 있는 압축 파일들 압축 풀기
for file in os.listdir(folder_path):
    zip_file_path = os.path.join(folder_path, file)
    if zipfile.is_zipfile(zip_file_path): 
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(zip_folder)

# csv 파일이 큰 순서대로 정렬
csv_files_sorted = sorted([os.path.join(zip_folder, f) for f in os.listdir(zip_folder) if f.endswith('.csv')],
                          key=lambda x: os.path.getsize(x),reverse=True)
csv_files_sorted

['./data/zip_data\\fulldata_07_24_04_P_└╧╣▌└╜╜─┴í.csv',
 './data/zip_data\\fulldata_07_24_05_P_╚▐░╘└╜╜─┴í.csv',
 './data/zip_data\\fulldata_07_24_01_P_░ⁿ▒ñ╜─┤τ.csv']

In [10]:
# 첫번째 데이터 불러오기 (데이터 크기가 커서 둘로 나눈뒤 합침)
chunk_size = 1000000
chunks = pd.read_csv(csv_files_sorted[0], encoding='cp949', encoding_errors='ignore', chunksize=chunk_size)
df1_1 = next(chunks)  # 첫 번째 청크(1,000,000 행) 읽기
df1_2 = pd.concat(chunks, ignore_index=True)  # 나머지 데이터 읽기

# 현재 운영하고 있는 매장만 가져와서 합치기 
df1_1.drop(df1_1[df1_1['영업상태명'] == '폐업'].index, inplace=True)
df1_2.drop(df1_2[df1_2['영업상태명'] == '폐업'].index, inplace=True)
df1 = pd.concat([df1_1, df1_2], ignore_index=True)

# 두번째 데이터 불러오기 
df2 = pd.read_csv(csv_files_sorted[1], encoding='cp949', encoding_errors='ignore')
df2.columns

# 세번째 데이터 불러오기 
df3 = pd.read_csv(csv_files_sorted[2], encoding='cp949', encoding_errors='ignore')
df3.columns

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '문화체육업종명', '문화사업자구분명', '지역구분명', '총층수', '주변환경명', '제작취급품목내용', '보험기관명',
       '건물용도명', '지상층수', '지하층수', '객실수', '건축연면적', '영문상호명', '영문상호주소', '선박총톤수',
       '선박척수', '선박제원', '무대면적', '좌석수', '기념품종류', '회의실별동시수용인원', '시설면적', '놀이기구수내역',
       '놀이시설수', '방송시설유무', '발전시설유무', '의무실유무', '안내소유무', '기획여행보험시작일자',
       '기획여행보험종료일자', '자본금', '보험시작일자', '보험종료일자', '부대시설내역', '시설규모',
       'Unnamed: 63'],
      dtype='object')

In [9]:
df4 = df3.copy()

df4 = df4[9000:15000]
df3.to_excel('dnjsqhsdpeldj.xlsx')

In [ ]:
# # 필요한 컬럼만 들고 오기 
# selected_columns = ['영업상태명', '소재지전체주소', '도로명전체주소', '사업장명', '최종수정시점','데이터갱신일자','업태구분명', '좌표정보(x)', '좌표정보(y)','위생업태명']

# df1 = df1[selected_columns]

# # 지역구 구분하기 
# df1['시도'] = df1['소재지전체주소'].apply(lambda x: x.split(' ', 1)[0] if pd.notna(x) else None)
# df1['지역구'] = df1['소재지전체주소'].apply(lambda x: x.split(' ')[1] if pd.notna(x) and len(x.split(' ')) > 1 else None)



# # 현재 운영하고 있는 매장만 가져오기
# df2.drop(df2[df2['영업상태명'] == '폐업'].index, inplace=True)

# # 필요한 컬럼만 들고 오기 
# df2 = df2[selected_columns]

# # 지역구 구분하기 
# df2['시도'] = df2['소재지전체주소'].apply(lambda x: x.split(' ', 1)[0] if pd.notna(x) else None)
# df2['지역구'] = df2['소재지전체주소'].apply(lambda x: x.split(' ')[1] if pd.notna(x) and len(x.split(' ')) > 1 else None)


# # 현재 운영하고 있는 매장만 가져오기
# df3.drop(df3[df3['영업상태명'] == '폐업'].index, inplace=True)

# # 필요한 컬럼만 들고 오기 
# df3 = df1[selected_columns]

# # 지역구 구분하기 
# df3['시도'] = df3['소재지전체주소'].apply(lambda x: x.split(' ', 1)[0] if pd.notna(x) else None)
# df3['지역구'] = df3['소재지전체주소'].apply(lambda x: x.split(' ')[1] if pd.notna(x) and len(x.split(' ')) > 1 else None)

# # 데이터 합치고 중복제거
# origin_df = pd.concat([df1, df2, df3], ignore_index=True).drop_duplicates()
# origin_df.head(10)

In [ ]:
# origin_df.info()

In [ ]:
# # '시도' 컬럼이 '서울특별시'면서 '지역구' 값이 '강남구'인 행만 필터링
# df = origin_df[(origin_df['시도'] == city ) & (origin_df['지역구'] == gu )]
# df.head()

# # csv 파일로 저장
# df.to_csv('filtered_data_{}{}.csv'.format(city,gu), index=False)

## 매장 존재 여부 확인

In [23]:
# 주소 + 매장명을 불러와서 네이버지도에 있는지/없는지 확인하는 함수 
def process_data(dataframe):
    global driver  # 전역 변수로 driver 사용

    # 진행 상황을 직접 출력
    for i in range(len(dataframe.loc[:,['사업장명','상세주소']])):
        name = dataframe['상세주소'][i] + ' ' + str(dataframe['사업장명'][i]) # 매장명이 숫자로만 되어 있는 경우 에러남
        driver.get('https://map.naver.com/p/search/{}'.format(name))
        time.sleep(2.2)  
    
        # 매장이 검색되는 경우 
        try:
            searchIframe = driver.find_element(By.ID,'searchIframe')
            driver.switch_to.frame(searchIframe)
            dataframe['검색여부'][i] = '가능'

        # 매장이 검색되지 않는 경우 
        except NoSuchElementException:
            pass  # continue 대신 pass를 사용하도록 수정

        return dataframe

## 그냥 식당이름이면 식당이름 , 식당이름 () + ~역점이면 식당이름 + ~역점, 식당이름 () 면 식당이름만 나오게 
def remove_parentheses_from_column(df, column_name):
    def remove_parentheses(input_string):
        start_index = input_string.find('(')
        end_index = input_string.find(')')
        
        if start_index != -1 and end_index != -1:  # '('와 ')'가 모두 존재하는 경우
            return input_string[:start_index] + input_string[end_index+1:]
        else:
            return input_string
    
    df[column_name] = df[column_name].apply(remove_parentheses)
    return df

# ~점으로 끝나는 매장명 ~점 부분 삭제하는 함수 
def remove_last_word(input_str):
    words = input_str.split()
    if len(words) > 1 and words[-1].endswith('점'):
        return ' '.join(words[:-1])
    else:
        return input_str

In [28]:
df = pd.read_csv('./data/filtered_data_서울특별시강남구.csv')
df.head(5)

,영업상태명,소재지전체주소,도로명전체주소,사업장명,최종수정시점,데이터갱신일자,업태구분명,좌표정보(x),좌표정보(y),위생업태명,시도,지역구
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,2023-09-22 11:33:55,2023-09-24 00:17:57,한식,202203.971845,444853.761458,한식,서울특별시,강남구
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,2023-09-22 10:20:03,2023-09-24 00:17:57,한식,202462.794327,443827.961945,한식,서울특별시,강남구
2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,2023-12-01 17:58:46,2023-12-03 00:17:36,기타,205130.591679,445590.096838,기타,서울특별시,강남구
3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,2023-12-01 17:01:54,2023-12-03 00:17:36,한식,203470.848439,447369.579852,한식,서울특별시,강남구
4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,2023-12-01 16:01:58,2023-12-03 00:17:36,일식,204447.401189,445478.066611,일식,서울특별시,강남구


In [29]:
df.isna().sum()

영업상태명       0
소재지전체주소     0
도로명전체주소    95
사업장명        0
최종수정시점      0
데이터갱신일자     0
업태구분명       1
좌표정보(x)    81
좌표정보(y)    81
위생업태명       1
시도          0
지역구         0
dtype: int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16384 entries, 0 to 16383
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영업상태명    16384 non-null  object 
 1   소재지전체주소  16384 non-null  object 
 2   도로명전체주소  16289 non-null  object 
 3   사업장명     16384 non-null  object 
 4   최종수정시점   16384 non-null  object 
 5   데이터갱신일자  16384 non-null  object 
 6   업태구분명    16383 non-null  object 
 7   좌표정보(x)  16303 non-null  float64
 8   좌표정보(y)  16303 non-null  float64
 9   위생업태명    16383 non-null  object 
 10  시도       16384 non-null  object 
 11  지역구      16384 non-null  object 
dtypes: float64(2), object(10)
memory usage: 1.5+ MB


In [31]:
df = df.iloc[6581:]

In [32]:
df

,영업상태명,소재지전체주소,도로명전체주소,사업장명,최종수정시점,데이터갱신일자,업태구분명,좌표정보(x),좌표정보(y),위생업태명,시도,지역구
6581,영업/정상,서울특별시 강남구 역삼동 669-16번지,서울특별시 강남구 논현로94길 11 (역삼동),민들레떡볶이,2012-12-07 15:32:30,2018-08-31 23:59:59,한식,203220.440823,444486.772022,한식,서울특별시,강남구
6582,영업/정상,서울특별시 강남구 신사동 661-18번지 지상1층,"서울특별시 강남구 압구정로50길 7 (신사동,지상1층)",일번지식당,2015-05-24 19:11:37,2018-08-31 23:59:59,한식,203343.164354,447278.968283,한식,서울특별시,강남구
6583,영업/정상,서울특별시 강남구 삼성동 156-5,서울특별시 강남구 삼성로 520 (삼성동),리칭,2023-12-22 15:15:24,2023-12-24 02:40:00,중국식,204877.603980,445165.526100,중국식,서울특별시,강남구
6584,영업/정상,서울특별시 강남구 삼성동 153-47번지,"서울특별시 강남구 삼성로100길 23-14, 지상1층 (삼성동)",더(The) 고깃집,2019-04-30 16:20:51,2019-05-02 02:40:00,한식,204992.888383,445365.922901,한식,서울특별시,강남구
6585,영업/정상,서울특별시 강남구 논현동 231-1번지,서울특별시 강남구 언주로115길 7 (논현동),손칼국수,2010-02-23 10:36:33,2018-08-31 23:59:59,한식,203122.416394,445323.659138,한식,서울특별시,강남구
...,...,...,...,...,...,...,...,...,...,...,...,...
16379,영업/정상,서울특별시 강남구 논현동 142-2 JS타워 지상1층,"서울특별시 강남구 강남대로 538, JS타워 지상1층 (논현동)",스타벅스 논현역사거리점,2023-02-20 15:40:13,2023-02-22 02:40:00,커피숍,201908.751105,445336.463782,커피숍,서울특별시,강남구
16380,영업/정상,서울특별시 강남구 논현동 119 포바강남타워,"서울특별시 강남구 학동로 343, 포바강남타워 지하1층 B103-1호 (논현동)",켈리 토스트 카페 강남구청역점,2023-09-19 10:11:01,2023-09-21 02:40:00,패스트푸드,203522.611749,446136.224972,패스트푸드,서울특별시,강남구
16381,영업/정상,서울특별시 강남구 신사동 664-6,"서울특별시 강남구 선릉로 845, 지상1층 6호 (신사동)",커피에 반하다,2022-12-15 15:39:26,2022-12-17 02:40:00,커피숍,203493.498594,447259.651525,커피숍,서울특별시,강남구
16382,영업/정상,서울특별시 강남구 논현동 218번지 지상1층,"서울특별시 강남구 언주로125길 7, 지상1층 (논현동)",CU 논현세원점,2018-11-05 10:57:06,2018-11-07 02:37:39,편의점,202990.979680,445632.581166,편의점,서울특별시,강남구


In [33]:
# 업태구분명 확인
unique_values = df['업태구분명'].unique()
unique_values

# 필요없는 카테고리(카페, 술, 출장요리 등등) 제거
제외 = ['전통찻집','호프/통닭','뷔페식','출장조리','정종/대포집/소주방','이동조리', '감성주점','까페','라이브카페','키즈카페','커피숍','편의점', '일반조리판매','아이스크림', '떡카페', '철도역구내', '푸드트럭', '과자점', '다방', '관광호텔']
for i in 제외 :
    df = df.drop(df[df['업태구분명'] == i].index)

# 필요없는 칼럼 제거 
df.drop(['최종수정시점','데이터갱신일자','위생업태명'],axis=1,inplace=True)


# 주소 추출 (도로명이 없는 경우 소재지 주소) 및 상세주소 열 추가
df['상세주소'] = df.apply(lambda row: ' '.join(row['도로명전체주소'].split(' ')[2:4]) if pd.notna(row['도로명전체주소']) else ' '.join(row['소재지전체주소'].split(' ')[2:4]), axis=1)

# 매장명 전처리 
df['사업장명_원본'] = df['사업장명']
df['사업장명'] = df['사업장명'].str.replace('(주)', '') # (주) 삭제   
remove_parentheses_from_column(df,'사업장명') # 매장명 간단하게 하는 함수 
df['사업장명'] = df['사업장명'].apply(remove_last_word) #~점 삭제하는 함수 

# 칼럼 생성 
df['검색여부'] = ''

df.reset_index(inplace=True, drop=True)
df.head()

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,사업장명_원본,검색여부
0,영업/정상,서울특별시 강남구 역삼동 669-16번지,서울특별시 강남구 논현로94길 11 (역삼동),민들레떡볶이,한식,203220.440823,444486.772022,서울특별시,강남구,논현로94길 11,민들레떡볶이,
1,영업/정상,서울특별시 강남구 신사동 661-18번지 지상1층,"서울특별시 강남구 압구정로50길 7 (신사동,지상1층)",일번지식당,한식,203343.164354,447278.968283,서울특별시,강남구,압구정로50길 7,일번지식당,
2,영업/정상,서울특별시 강남구 삼성동 156-5,서울특별시 강남구 삼성로 520 (삼성동),리칭,중국식,204877.603980,445165.526100,서울특별시,강남구,삼성로 520,리칭,
3,영업/정상,서울특별시 강남구 삼성동 153-47번지,"서울특별시 강남구 삼성로100길 23-14, 지상1층 (삼성동)",더 고깃집,한식,204992.888383,445365.922901,서울특별시,강남구,"삼성로100길 23-14,",더(The) 고깃집,
4,영업/정상,서울특별시 강남구 논현동 231-1번지,서울특별시 강남구 언주로115길 7 (논현동),손칼국수,한식,203122.416394,445323.659138,서울특별시,강남구,언주로115길 7,손칼국수,


In [ ]:
# df.to_excel('./data/매장명_전처리.xlsx')


In [35]:
len(df)

6304

In [34]:
df.isna().sum()

영업상태명       0
소재지전체주소     0
도로명전체주소    43
사업장명        0
업태구분명       1
좌표정보(x)    36
좌표정보(y)    36
시도          0
지역구         0
상세주소        0
사업장명_원본     0
검색여부        0
dtype: int64

# 셀레니움

In [36]:
# 검색할 데이터 수 지정 
sample_n = 10

# # 1. 인덱스 기준 200개 추출
# df_test = df.iloc[:sample_n]

# 2. 랜덤으로 100개 추출
df_test = df.sample(n=sample_n, random_state=42)
df_test.reset_index(inplace=True, drop=True)
df_test.head(10)

df_test

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,사업장명_원본,검색여부
0,영업/정상,서울특별시 강남구 역삼동 653-4번지 일진스포월드 지상2층,"서울특별시 강남구 언주로 563 (역삼동,일진스포월드 지상2층)",제이브라운,패스트푸드,203311.617902,445074.121017,서울특별시,강남구,언주로 563,제이브라운,
1,영업/정상,서울특별시 강남구 역삼동 656-18번지,서울특별시 강남구 논현로106길 52 (역삼동),아이스벅,경양식,203337.947644,444981.282640,서울특별시,강남구,논현로106길 52,아이스벅,
2,영업/정상,서울특별시 강남구 역삼동 732-17 지상1층 102호,"서울특별시 강남구 역삼로37길 3, 지상1층 102호 (역삼동)",유메,일식,203737.617053,443992.307465,서울특별시,강남구,"역삼로37길 3,",유메(ゆめ),
3,영업/정상,서울특별시 강남구 개포동 1238-7번지,"서울특별시 강남구 개포로 216, 지상1층 (개포동)",배스킨라빈스,패스트푸드,204037.439263,441806.241681,서울특별시,강남구,"개포로 216,",배스킨라빈스,
4,영업/정상,서울특별시 강남구 삼성동 160-18번지,서울특별시 강남구 봉은사로 610 (삼성동),꽃지삼성,한식,205381.162419,445762.980930,서울특별시,강남구,봉은사로 610,꽃지삼성,
5,영업/정상,서울특별시 강남구 삼성동 143-34 1층,"서울특별시 강남구 삼성로91길 38 (삼성동,1층)",선릉고기집,경양식,204628.130620,444924.718059,서울특별시,강남구,삼성로91길 38,선릉고기집,
6,영업/정상,서울특별시 강남구 청담동 49-8번지,"서울특별시 강남구 도산대로 514, 2동 1층 11,19호 (청담동)",부자소곱창,한식,204283.727260,446864.739328,서울특별시,강남구,"도산대로 514,",부자소곱창,
7,영업/정상,서울특별시 강남구 신사동 589-9 알파빌딩지상1층,"서울특별시 강남구 논현로152길 23 (신사동,알파빌딩지상1층)",룽청,중국식,202596.595812,446584.403089,서울특별시,강남구,논현로152길 23,룽청,
8,영업/정상,서울특별시 강남구 압구정동 494-0번지 지하1층,"서울특별시 강남구 압구정로 343 (압구정동,지하1층)",슐라스키델리,패스트푸드,203470.848439,447369.579852,서울특별시,강남구,압구정로 343,슐라스키델리,
9,영업/정상,서울특별시 강남구 신사동 648-24번지,"서울특별시 강남구 압구정로46길 71, 지상1층 (신사동)",클랩피자,기타,203164.773812,446874.689705,서울특별시,강남구,"압구정로46길 71,",클랩피자 청담점,


In [37]:
# 데이터프레임 나누기
batch_size = 100
num_batches = len(df) // batch_size

df_parts = [df.iloc[i * batch_size: (i + 1) * batch_size].copy() for i in range(num_batches)]

# 마지막 데이터프레임은 나머지 데이터 포함
df_parts[-1] = df.iloc[num_batches * batch_size:].copy()

# 각 부분 데이터프레임에 새로운 인덱스 추가 및 원래 인덱스 저장
for i, df_part in enumerate(df_parts):
    df_part['index_original'] = df_part.index  # 원래 인덱스 저장
    df_part.reset_index(inplace=True, drop=True)  # 새로운 인덱스 추가


# 데이터프레임 몇개야 
n = len(df_parts)

df_parts[2]

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,사업장명_원본,검색여부,index_original
0,영업/정상,서울특별시 강남구 수서동 502-1번지,서울특별시 강남구 광평로 220 (수서동),한단고기,한식,208515.278297,442505.890429,서울특별시,강남구,광평로 220,한단고기,,200
1,영업/정상,서울특별시 강남구 삼성동 153-6 지하1층,"서울특별시 강남구 삼성로100길 23-11 (삼성동,지하1층)",루아,한식,204962.146645,445395.657047,서울특별시,강남구,삼성로100길 23-11,루아,,201
2,영업/정상,서울특별시 강남구 역삼동 667-6번지,"서울특별시 강남구 논현로 540, 지상1층 (역삼동)",빨간모자피자,기타,203069.480000,444698.630000,서울특별시,강남구,"논현로 540,",빨간모자피자 역삼점,,202
3,영업/정상,서울특별시 강남구 삼성동 159번지 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 동측광장층 (삼성동)",레시피뱅크,경양식,205130.591679,445590.096838,서울특별시,강남구,"영동대로 513,",레시피뱅크,,203
4,영업/정상,서울특별시 강남구 신사동 616-2번지 지상1층 107호,"서울특별시 강남구 압구정로 224, 지상1층 107호 (신사동)",압구정와플반트,패스트푸드,202772.968268,447366.539357,서울특별시,강남구,"압구정로 224,",압구정와플반트,,204
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,영업/정상,서울특별시 강남구 신사동 533-9,"서울특별시 강남구 가로수길 57-6, 지상1층 102호 (신사동)",필라델피아 가로수,기타,201898.911558,446628.752845,서울특별시,강남구,"가로수길 57-6,",필라델피아 가로수,,295
96,영업/정상,서울특별시 강남구 논현동 111-23 지상1층,"서울특별시 강남구 선릉로 713, 지상1층 (논현동)",두레반 청담아구찜,일식,203515.057348,446257.804816,서울특별시,강남구,"선릉로 713,",두레반 청담아구찜,,296
97,영업/정상,서울특별시 강남구 신사동 652-7번지 젤로빌딩,"서울특별시 강남구 도산대로49길 10-6, 젤로빌딩 지상1층 (신사동)",도산분식,한식,203279.713577,446829.052325,서울특별시,강남구,"도산대로49길 10-6,",도산분식(도산공원점),,297
98,영업/정상,서울특별시 강남구 수서동 713번지 수서현대벤쳐빌,"서울특별시 강남구 밤고개로1길 10, 수서현대벤쳐빌 지하2층 245호 (수서동)",청년떡볶이,기타,208937.760652,442873.588040,서울특별시,강남구,"밤고개로1길 10,",청년떡볶이 수서역점,,298


In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager




# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정

# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()


# 저는 에러나서 이걸로 사용, 이게 이러나면 위에 코드 사용 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-dev-shm-usage:dev/shm')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)


In [ ]:
# # 네이버 매장 있나 없나 함수로 확인하기. 
# for i in range(len(df_parts)) :
#     chdeck_data = process_data(df_parts[i])
#     chdeck_data.to_excel(f'.data/매장존재확인/매장존재확인{i}.xlsx')


In [40]:
# # # 크롬 드라이버 최신 버전 설정
# # service = ChromeService(executable_path=ChromeDriverManager().install())
        
# # chrome driver
# driver = webdriver.Chrome(service=service, options=options) 


# 저는 에러나서 이걸로 사용, 이게 이러나면 위에 코드 사용 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-dev-shm-usage:dev/shm')
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
# driver = webdriver.Chrome(service=service, options=options) 

driver = webdriver.Chrome(ChromeDriverManager().install())

# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)


# tqdm 데코레이터를 사용하여 반복문 진행 상황을 표시
j = 0
df_parts[j]

for i in tqdm(range(len(df_parts[j].loc[:,['사업장명','상세주소']]))):

    global driver  # 전역 변수로 driver 사용

    name = df_parts[j]['상세주소'][i] + ' ' + str(df_parts[j]['사업장명'][i]) # 매장명이 숫자로만 되어 있는 경우 에러남
    driver.get('https://map.naver.com/p/search/{}'.format(name))
    time.sleep(2.2)  
    
    # 매장이 검색되는 경우 
    try :
        searchIframe = driver.find_element(By.ID,'searchIframe')
        driver.switch_to.frame(searchIframe)
        df_parts[j]['검색여부'][i] = '가능'


    # 매장이 검색되지 않는 경우 
    except NoSuchElementException :
        continue

driver.quit()
df_parts[j].to_excel(f'./data/매장존재확인/매장존재확인{j}.xlsx')
j += 1


100%|██████████| 100/100 [06:51<00:00,  4.11s/it]


어디까지 되나 하는 중 300개씩 21에서 ooutofmemory, 300개당 12분에서 20분으로 늘 